In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sqlalchemy import func
from tqdm import tqdm

print(os.getcwd())
os.chdir('../')
print(os.getcwd())
from src.buildDataset import build_static_dataset
from src.sqlstore.db import get_session
from src.sqlstore.match import SQLMatch, SQLParticipantStats, SQLParticipant
from src.sqlstore.timeline import SQLTimeline, SQLEvent, SQLKillEvent, SQLFrame, SQLParticipantFrame, \
    SQLTimelineDamageDealt, SQLTimelineDamageReceived
from src.utils import separateMatchID

C:\Users\morit\Documents\Informatik\Projekte\LeagueOfLegendsWinPrediction\Exploration
C:\Users\morit\Documents\Informatik\Projekte\LeagueOfLegendsWinPrediction


In [2]:
def build_frame_dataset(size: int, save: bool):
    with get_session() as session:
        matches = session.query(SQLMatch).order_by(func.random()).limit(size).all()
        matchIds = []
        frameData = []
        # iterate over matches
        for match in tqdm(matches):
            platformId, gameId = separateMatchID(match.matchId)
            # calculate winning team
            participantId = session.query(SQLParticipant.id).filter(SQLParticipant.matchId == match.matchId,
                                                                    SQLParticipant.participantId == 1).scalar()
            winning_team = int(session.query(SQLParticipantStats.win
                                             ).filter(SQLParticipantStats.participantId == participantId
                                                      ).scalar())
            # get timeline and frames for that match
            timeline = session.query(SQLTimeline).filter_by(platformId=platformId, gameId=gameId).first()
            frames = session.query(SQLFrame).filter_by(timelineId=timeline.id).all()
            participantFrameData = []
            # iterate over frames
            for frame in frames:
                matchIds.append((match.matchId, frame.id))
                frameDict = {'timestamp': frame.timestamp, 'winning_team': winning_team}
                participantFrames = session.query(SQLParticipantFrame).filter_by(frameId=frame.id).all()
                for i, participantFrame in enumerate(participantFrames):
                    participantFrameTrainingData = participantFrame.get_training_data()
                    participantFrameData.append(participantFrameTrainingData)
                    participantFrameDict = {f'participant{i + 1}_{k}': v for k, v in
                                            participantFrameTrainingData.items()}
                    frameDict.update(participantFrameDict)
                frameData.append(frameDict)
        index = pd.MultiIndex.from_tuples(matchIds, names=['matchId', 'frameId'])
        print(f"len of framedata: {len(frameData)}")
        print(f"shape of index: {index.shape}")
        dfTimelines = pd.DataFrame(frameData, index=index)
        if save:
            dfTimelines.to_pickle('data/raw/timelines.pkl')
        return dfTimelines


In [3]:
data = build_frame_dataset(size=1000, save=True)
data

100%|██████████| 1000/1000 [2:17:19<00:00,  8.24s/it] 


len of framedata: 28301
shape of index: (28301,)
len of matchIds: 28301


timestamp  winning_team  participant1_participantId  \
matchId         frameId                                                        
EUW1_6621423080 23043            0             0                           1   
                23044        60006             0                           1   
                23045       120032             0                           1   
                23046       180088             0                           1   
                23047       240118             0                           1   
...                            ...           ...                         ...   
EUW1_6626278513 406763     1320428             0                           1   
                406764     1380460             0                           1   
                406765     1440483             0                           1   
                406766     1500510             0                           1   
                406767     1506815             0                           1   

                         participant1_abilityHaste  participant1_abilityPower  \
matchId         frameId                                                         
EUW1_6621423080 23043                            0                          0   
                23044                            0                          0   
                23045                            0                          0   
                23046                            0                          0   
                23047                            0                          0   
...                                            ...                        ...   
EUW1_6626278513 406763                           0                          9   
                406764                           0                          9   
                406765                           0                          9   
                406766                           0                          9   
                406767                           0                          9   

                         participant1_armor  participant1_armorPen  \
matchId         frameId                                              
EUW1_6621423080 23043                    36                      0   
                23044                    42                      0   
                23045                    42                      0   
                23046                    48                      0   
                23047                    48                      0   
...                                     ...                    ...   
EUW1_6626278513 406763                  205                      0   
                406764                  205                      0   
                406765                  205                      0   
                406766                  205                      0   
                406767                  205                      0   

                         participant1_armorPenPercent  \
matchId         frameId                                 
EUW1_6621423080 23043                               0   
                23044                               0   
                23045                               0   
                23046                               0   
                23047                               0   
...                                               ...   
EUW1_6626278513 406763                              0   
                406764                              0   
                406765                              0   
                406766                              0   
                406767                              0   

                         participant1_attackDamage  participant1_attackSpeed  \
matchId         frameId                                                        
EUW1_6621423080 23043                           25                       100   
                23044                

In [4]:
def build_timeline_dataset(size: int, save: bool) -> pd.DataFrame:
    with get_session() as session:
        matches = session.query(SQLMatch).order_by(func.random()).limit(size).all()
        matchIds = []
        timelineIds = []
        for match in matches:
            matchIds.append(match.matchId)
            platformId, gameId = separateMatchID(match.matchId)
            timeline = session.query(SQLTimeline).filter_by(platformId=platformId, gameId=gameId).first()
            timelineIds.append(timeline.id)
            frames = session.query(SQLFrame).filter_by(timelineId=timeline.id).all()
            eventData = []
            frameIds = []
            eventIds = []
            for frame in frames:
                frameIds.append(frame.id)
                events = session.query(SQLEvent).filter_by(frameId=frame.id).all()
                # append all events in that frame to the eventData list
                for event in events:
                    eventIds.append(event.id)
                    eventData.append(event.get_training_data())
                killevents = session.query(SQLKillEvent).filter_by(frameId=frame.id).all()
                for killevent in killevents:
                    eventData.append(killevent.get_training_data())
                for participantFrame in frame.participantframe:
                    participantFrameData = participantFrame.get_training_data()
                    participantFrameData['timestamp'] = frame.timestamp
                    eventData.append(participantFrameData)
            eventData.sort(key=lambda x: x['timestamp'])
            print(eventData)
            df = pd.DataFrame(eventData)

            frameIds = []
            eventIds = []
            frame_timestamps = []
            event_Ids_timestamps = []
            participantFrames = []
            for frame in frames:
                frameIds.append(frame.id)
                frame_timestamps.append(frame.timestamp)
                for event in frame.events:
                    eventIds.append(event.id)
                    event_Ids_timestamps.append((event.id, event.timestamp))
                for participantFrame in frame.participantframe:
                    participantFrames.append(participantFrame.get_training_data())
            participants_list = [x for x in range(1, 11)]
            index_tuples = [(timeline.id, frame.id, participantId) for frame in frames for participantId in
                            participants_list]
            participantFrame_columns = participantFrames[0].keys()
            #print(participantFrame_columns)
            #print(index_tuples)
            index = pd.MultiIndex.from_tuples(index_tuples, names=['timelineId', 'frameId', 'participantId'])
            df = pd.DataFrame(participantFrames, index=index, columns=participantFrame_columns)
            return df
        #data = pd.DataFrame(zip(matchIds, timelineIds), columns=['matchId', 'timelineId'])

    return data

In [5]:
data = build_timeline_dataset(size=100, save=False)
data

[{'eventId': 0, 'timestamp': 0, 'type': 'PAUSE_END', 'participantId': None, 'itemId': None, 'skillSlot': None, 'creatorId': None, 'teamId': None, 'afterId': None, 'beforeId': None, 'wardType': None}, {'participantId': 1, 'abilityHaste': 0, 'abilityPower': 0, 'armor': 31, 'armorPen': 0, 'armorPenPercent': 0, 'attackDamage': 25, 'attackSpeed': 100, 'bonusArmorPenPercent': 0, 'bonusMagicPenPercent': 0, 'ccReduction': 0, 'cooldownReduction': 0, 'health': 600, 'healthMax': 600, 'healthRegen': 0, 'lifesteal': 0, 'magicPen': 0, 'magicPenPercent': 0, 'magicResist': 32, 'movementSpeed': 345, 'omnivamp': 0, 'physicalVamp': 0, 'power': 280, 'powerRegen': 0, 'spellVamp': 0, 'currentGold': 500, 'magicDamageDone': 0, 'magicDamageDoneToChampions': 0, 'magicDamageTaken': 0, 'physicalDamageDone': 0, 'physicalDamageDoneToChampions': 0, 'physicalDamageTaken': 0, 'totalDamageDone': 0, 'totalDamageDoneToChampions': 0, 'totalDamageTaken': 0, 'trueDamageDone': 0, 'trueDamageDoneToChampions': 0, 'trueDamageTa

participantId  abilityHaste  abilityPower  \
timelineId frameId participantId                                              
8222       231430  1                          1             0             0   
                   2                          2             0             0   
                   3                          3             0             0   
                   4                          4             0             0   
                   5                          5             0             0   
...                                         ...           ...           ...   
           231441  6                          6             0             0   
                   7                          7             0           121   
                   8                          8             0             0   
                   9                          9             0             0   
                   10                        10             0            20   

                                  armor  armorPen  armorPenPercent  \
timelineId frameId participantId                                     
8222       231430  1                 31         0                0   
                   2                 29         0                0   
                   3                 20         0                0   
                   4                 24         0                0   
                   5                 28         0                0   
...                                 ...       ...              ...   
           231441  6                 68         0               10   
                   7                 59         0                0   
                   8                 56         0                0   
                   9                 56         0                0   
                   10                66         0                0   

                                  attackDamage  attackSpeed  \
timelineId frameId participantId                              
8222       231430  1                        25          100   
                   2                        25          100   
                   3                        25          100   
                   4                        25          100   
                   5                        25          100   
...                                        ...          ...   
           231441  6                       153          117   
                   7                        75          110   
                   8                       120          202   
                   9                       142          123   
                   10                       71          113   

                                  bonusArmorPenPercent  bonusMagicPenPercent  \
timelineId frameId participantId                                               
8222       231430  1                                 0                     0   
                   2                                 0                     0   
                   3                                 0                     0   
                   4                                 0                     0   
                   5                                 0                     0   
...                                                ...                   ...   
           231441  6                                 0                     0   
                   7                                 0                     0   
                   8                                 0                     0   
                   9                                 0                     0   
                   10                                0                     0   

                                  ...  trueDamageTaken  goldPerSecond  \
timelineId frameId participantId  ...                                   
8222       231430  1              ...                0              0   
               